This notebook is used to generate the loss function arrays given results csv

In [1]:
import pandas as pd
import numpy as np

In [8]:
dataset = 'mnist'
method = 'dpsgd-g-a'
# before processing
raw_result=True
fairness_var = 'tau'
coverage = False
loss_function_file_path = f"./{dataset}/{method}/"

In [18]:
df = pd.read_csv(f'{loss_function_file_path}results.csv')

In [19]:
if raw_result and method == 'dpsgd-g-a':
    df['achieved_fairness_gap'] = abs((df['accuracy_2']-0.93) - (df['accuracy_8']-0.86))

In [20]:
if coverage:
    df = df[['epsilon', fairness_var, 'achieved_epsilon', 'achieved_fairness_gap', 'accuracy', 'coverage']]
else:
    df = df[['epsilon', fairness_var, 'achieved_epsilon', 'achieved_fairness_gap', 'accuracy']]
    if raw_result:
        # save back
        df.to_csv(f'{loss_function_file_path}results.csv', index=False)

In [14]:
# builder loss
acc = np.array(df['accuracy'])
# take the percentage version of accuracy for consistency
if max(acc) <= 1:
    acc = 100 * acc
acc = acc.reshape((len(acc), 1))

np.save(f'{loss_function_file_path}builder_loss_acc.npy', acc)

if coverage:
    cov = np.array(df['coverage'])
    # take the percentage version of coverage for consistency
    if max(cov) <= 1:
        cov = 100 * cov
    cov = cov.reshape((len(cov), 1))

    np.save(f'{loss_function_file_path}builder_loss_cov.npy', cov)

In [15]:
# fairness loss
fairness = np.array(df['achieved_fairness_gap'])
fairness = fairness.reshape((len(fairness), 1))

np.save(f'{loss_function_file_path}fairness_loss.npy', fairness)

In [16]:
# privacy loss
privacy = np.array(df['achieved_epsilon'])
privacy = privacy.reshape((len(privacy), 1))

np.save(f'{loss_function_file_path}privacy_loss.npy', privacy)

In [17]:
# input parameters
fairness_input = np.array(df[fairness_var])
fairness_input = fairness_input.reshape((len(fairness_input), 1))

privacy_input = np.array(df['epsilon'])
privacy_input = privacy_input.reshape((len(privacy_input), 1))
inputs = np.concatenate((privacy_input, fairness_input), axis = 1)

np.save(f'{loss_function_file_path}priv_fair_values.npy', inputs)